# Editor Timeline Basics (Hello World)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/editor/feature/timeline_basic.ipynb)

---

This is a runnable, end-to-end walkthrough of VideoDB **Editor**.

In this notebook, we’ll build the simplest possible composition: **one Timeline → one Track → one Clip (wrapping one VideoAsset)**, then generate a stream URL and play it.

**What you’ll do:**
- Install the Editor SDK
- Connect to VideoDB securely
- Upload one video asset
- Compose a timeline using Timeline/Track/Clip/VideoAsset
- Render and preview the result

---
## 📦 Step 1: Install dependencies

Lets install the VideoDB SDK

In [ ]:
%pip -q install videodb

  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Run the next cell and enter your `VIDEO_DB_API_KEY` when prompted.

In [ ]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---
## 📦 Step 3: Connect to a collection

A **collection** is where your uploaded assets live (videos, images, audio).
We’ll upload a video next and use its `id` inside Editor.

In [ ]:
coll = conn.get_collection()

---
## 📦 Step 4: Upload a video asset

Editor works with **assets stored in VideoDB**.
Here we upload a YouTube URL as a VideoDB video asset. The returned object has an `id` we’ll pass into `VideoAsset(id=...)`.

> Tip: When you re-run a notebook, you can skip re-uploading and fetch an existing asset by ID instead (see the commented snippet below).

In [ ]:
# Upload any asset by url
video = coll.upload(url="https://www.youtube.com/watch?v=RB9nyUyNI2s")
print("Uploaded video asset:", video.id)

# Optional re-run pattern (don’t upload again):
# video = coll.get_video("")

Uploaded video asset: m-z-019b49b9-8618-7510-9cc7-6e0133c88bf1


---
## 📦 Step 5: Import Editor building blocks (and the mental model)

Editor composition always follows the same shape:

- **Timeline**: the global canvas (resolution + background). It represents the *final video*.
- **Track**: a layer on the timeline. Tracks stack — later tracks render on top of earlier ones.
- **Clip**: a container that controls *when* something appears and for *how long* (duration).
- **Asset**: the raw media/content (video/audio/image/text/captions).

In the next cell we’ll import just what we need for Hello World: `Timeline`, `Track`, `Clip`, and `VideoAsset` (plus `play_stream` to preview the output).

In [ ]:
import json
from videodb import play_stream
from videodb.editor import Timeline, Track, Clip, VideoAsset

---
## 📦 Step 6: Create the Timeline (global canvas)

A `Timeline` represents the final output video.
We start by creating it from the VideoDB connection (`conn`).

In [ ]:
timeline = Timeline(conn)

---
## 📦 Step 7: Set resolution and background

These are **global canvas settings**:
- `resolution` defines the output frame size
- `background` is the solid color shown anywhere nothing else covers the canvas


In [ ]:
timeline.background = "#808080"
timeline.resolution = "600x1060"

---
## 📦 Step 8: Create a VideoAsset (points at uploaded media)

The upload step gave us a `video` object with a VideoDB media `id`.
A `VideoAsset` is a lightweight reference to that media, plus a few source-level controls.

Here we set:
- `id=video.id` to reference the uploaded video
- `start=0` (trim from the start of the source)
- `volume=1` (keep original volume)

In [ ]:
video_asset = VideoAsset(
    id=video.id,
    start=0,
    volume=1,
 )

---
## 📦 Step 9: Wrap the asset in a Clip (timeline timing)

A `Clip` is the container that controls how the asset behaves **on the timeline**.
For Hello World, we only need one key field: `duration`.

Think of it as: “play this asset for N seconds in the final video.”

In [ ]:
clip = Clip(asset=video_asset, duration=10)

---
## 📦 Step 10: Create a Track and place the clip at a start time

A `Track` is a layer on the timeline.
You add clips by telling the track **when** they should start in the final video.

This is also where the “double start” idea begins (we’ll go deep later):
- `VideoAsset(start=...)` trims the **source** media
- `track.add_clip(start=...)` places the clip on the **timeline**

In [ ]:
track = Track()
track.add_clip(0, clip)

---
## 📦 Step 11: Add the Track to the Timeline

A timeline can contain multiple tracks. They render as layers.
Since we have one track, we just attach it to the timeline.

In [ ]:
timeline.add_track(track)

---
## 📦 Step 12: (Optional) Inspect the timeline JSON

Editor ultimately sends a JSON description of your timeline to the render service.
Printing it is a great way to debug what you *think* you built vs what Editor will render.

In [ ]:
print(json.dumps(timeline.to_json()))

{"timeline": {"background": "#808080", "resolution": "600x1060", "tracks": [{"clips": [{"start": 0, "clip": {"asset": {"type": "video", "id": "m-z-019b49b9-8618-7510-9cc7-6e0133c88bf1", "start": 0, "volume": 1, "crop": {"top": 0, "right": 0, "bottom": 0, "left": 0}}, "duration": 10, "effect": null, "scale": 1, "opacity": 1, "fit": "crop", "position": "center", "offset": {"x": 0, "y": 0}, "z_index": 0}}], "z_index": 0}]}}


---
## 📦 Step 13: Generate the final stream URL

This is the render step: Editor takes your timeline description and generates a playable stream URL.

In [ ]:
stream_url = timeline.generate_stream()
print(stream_url)

https://play.videodb.io/v1/53809c7d-d53b-487e-8741-9905faa41946.m3u8


---
## 📦 Step 14: Play the stream

Finally, we’ll preview the rendered result directly from the generated URL.

In [ ]:
play_stream(stream_url)

---
## ✅ Wrap-up

You’ve just created a complete Editor composition end-to-end:

- Installed the Editor SDK
- Connected to VideoDB securely
- Uploaded a video and captured its `id`
- Built a `Timeline` (global canvas) with `resolution` and `background`
- Added a `Track`, placed a `Clip` on it, and wrapped your media as a `VideoAsset`
- Rendered the result with `timeline.generate_stream()` and previewed it via `play_stream(...)`
